In [1]:
import heapq
import numpy as np
from typing import Optional, List, Tuple
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from collections import namedtuple
print(np.__version__)
import random
import copy
import time

import torch
print(torch.__version__)

import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import trange, tqdm
import wandb
from omegaconf import OmegaConf, DictConfig

# Set precision to 3 decimal places
np.set_printoptions(precision=3, suppress=True)

import pickle

from discrete_maze.maze import Maze
from discrete_maze.search_algorithm import Node


2.1.3
2.5.1+cu124


In this notebook, I create a dataset of optimal demonstrations for 


In [2]:
# Define the configuration using OmegaConf
cfg = OmegaConf.create({
    "name": "maze_4to50_0to0p3_500000", # Will be used as the dataset name
    "dataset_generation": {
        "num_samples": 500000,
    },
    "maze": {
        "width": {"min": 4, "max": 50},
        "height": {"min": 4, "max": 50},
        "cell_occupancy_prob": {"min": 0, "max": 0.3},
        "max_steps": "ShortestPath", # Use this to set the max steps to the shortest path between source and target * 2
        # "max_steps": "L1SourceTarget", # Use this to set the max steps to the L1 distance between source and target * 2
        # To set paramters to constant values, use a float
        # "width": 4,
        # "height": 4,
        # "cell_occupancy_prob": 0,
        # "max_steps": 5, 
    },
    "model": {
        "num_resBlocks": 4,
        "num_filters": 64,
        "history_length": 8, # Number of current and previous states to consider, 1 for current state only
    },
})

In [ ]:

def generate_maze_dataset(file_name, maze_cfg, num_episodes, seed: Optional[int] = None):
    """Generate a dataset of mazes and save it to a file. Does not save the observations/actions etc."""
    if seed is not None:
        np.random.seed(seed)
    else:
        seed = np.random.randint(0, 2**32 - 1)
        np.random.seed(seed)

    print(f"Seed: {seed}")

    maze_params = Maze.generate_maze_params(num_episodes, maze_cfg=maze_cfg)
    mazes = [Maze(*params) for params in tqdm(maze_params, desc="Generating mazes")]
    episodes = [
        {
            "map": maze.map,
            "source": maze.source,
            "target": maze.target,
            "cell_occupancy_prob": maze.cell_occupancy_prob,
            "max_steps": maze.max_steps,
            "shortest_path": maze.shortest_path,
        } for maze in mazes
    ]

    dataset = {
        "episodes": episodes,
        "dataset_maze_cfg": OmegaConf.to_container(maze_cfg, resolve=True, throw_on_missing=True),
        "seed": seed,
    }

    file_path = f"datasets/{file_name}.pkl"
    with open(file_path, 'wb') as f:
        pickle.dump(dataset, f)
    print(f"Dataset saved to {file_path}")



In [4]:
# generate_maze_dataset("maze_4to50_0to0p3_50000", cfg.maze, 50000)
# generate_maze_dataset("maze_4to50_0to0p3_10000", cfg.maze, 10000)
# generate_maze_dataset("maze_4to50_0to0p3_100", cfg.maze, 100)
generate_maze_dataset(cfg.name, cfg.maze, cfg.dataset_generation.num_samples)

Seed: 4208305061
Unsolvable maze 20. Regenerating...
Unsolvable maze 20. Regenerating...
Unsolvable maze 20. Regenerating...
Unsolvable maze 40. Regenerating...
Dataset saved to datasets/maze_4to50_0to0p3_500000.pkl


## Tests for dataset generation

In [5]:
# Test that the seed makes the generated datasets reproducible
# from deepdiff import DeepDiff
# generate_maze_dataset("maze_4to50_0to0p3_10_a", cfg.maze, 100, seed=0)
# generate_maze_dataset("maze_4to50_0to0p3_10_b", cfg.maze, 100, seed=0)

# with open("datasets/maze_4to50_0to0p3_10_a.pkl", "rb") as file:
#     loaded_data_a = pickle.load(file)
# with open("datasets/maze_4to50_0to0p3_10_b.pkl", "rb") as file:
#     loaded_data_b = pickle.load(file)

# diff = DeepDiff(loaded_data_a, loaded_data_b, ignore_order=True)

# if not diff:
#     print("The objects are identical.")
# else:
#     print("The objects differ:", diff)